In [27]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [28]:
src_train_parquet = 'ke_test_data/train.parquet'
src_test_parquet = 'ke_test_data/test.parquet'
src_categories_tree = 'ke_test_data/categories_tree.csv'

In [29]:
categories_tree = pd.read_csv(src_categories_tree)
train = pq.read_table(src_train_parquet).to_pandas()
test = pq.read_table(src_test_parquet).to_pandas()

In [30]:
def get_text_data_func(x, sep=' '):
    # дергает текстовые данные из входящего датафрейма
    result = x.copy()
    result['text'] = ''
    for i, key in enumerate(model_params['text']):
        if i != 0:
            result['text'] = result['text'] + sep
        result['text'] = result['text'] + result[key].astype(str)

    return result['text'].to_numpy()

def get_numeric_data_func(x):
    # дергает числовые данные из входящего датафрейма
    x = x[model_params['numeric']].copy()
    column = model_params['numeric'][0]
    x[column] = x[column].apply(hist_edges_mapper)
    return pd.DataFrame(x['rating'].to_list()).to_numpy()

In [31]:
# дерево каталогов
import networkx as nx
DG = nx.DiGraph()


for k, v in tqdm(categories_tree.iterrows(), desc='Строим дерево'):
    DG.add_edge(v['parent_id'], v['id'])


def all_predcestors(g, id, max_len=7):
    # циклично запрашиваем предков, пока их не будет совсем
    all_preds = [id]
    while True:
        preds = list(g.predecessors(id))
        if len(preds) == 0:
            break
        all_preds.append(preds[0])
        id = all_preds[-1]

    # доводим до максимального размера, чтобы вмещалось в tensor/array
    # паддинг -1 (0 занят нашим каталогом, мы позже их закодируем, но всеравно...)
    if len(all_preds) < max_len:
        all_preds = [-1] * (max_len - len(all_preds)) + all_preds
    return all_preds[::-1]

catalog_cnt = len(DG.nodes)

print(f'Число нод в дереве: {catalog_cnt}')

Строим дерево: 3370it [00:00, 14727.65it/s]

Число нод в дереве: 3373


In [32]:
# прогресс бары внутри пайплайна
num_pbar = None
text_pbar = None

def start_num_pbar(x):
    global num_pbar
    num_pbar = tqdm(total=len(x), desc='Nums processing')
    return x


def shut_num_pbar(x):
    global num_pbar
    num_pbar.close()
    return x


def start_text_pbar(x):
    global text_pbar
    text_pbar = tqdm(total=len(x), desc='Text processing')
    return x


def shut_text_pbar(x):
    global text_pbar
    text_pbar.close()
    return x

In [33]:
# диапазон значений рейтингов [0:5] разбили на 6 равных диапазонов
hist_bin_edges = list(np.histogram_bin_edges(train['rating'], bins=6))

def hist_edges_mapper(val, bin_borders=hist_bin_edges):
    # мэпим значения в диапазоны гистограммы
    resval = []
    for i in range(len(bin_borders)-1):
        if bin_borders[i] <= val < bin_borders[i+1]:
            resval.append(1)
        elif i == len(bin_borders) - 2:
            if val == bin_borders[i+1]:
                resval.append(1)
            else:
                resval.append(0)
        else:
            resval.append(0)

    global num_pbar
    num_pbar.update(1)

    return resval

In [34]:
# свой токенайзер + стэммер  (так мы уменьшим размерность с 80к до 50к уникальных токенов
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re

regex = r"[\s\|$:*!?#:,.(){}\"\'\$\^\/\\\[\]\-+”“%¡¿&;«»`\d]"

punctuation = set(['\t','\n','\\', '', 'none'])

nltk.download("stopwords")
nltk.download('punkt')

russian_stopwords = set(stopwords.words("russian"))  # список стоп слов  -- не подошел.
r_num_ponctuation = set(punctuation)  # список символов (не букв и не цифр)


class StemTokenizer:
    # Стэммер вычленяет корни слов
    def __init__(self):
        self.stemer = SnowballStemmer('russian')
    def __call__(self, doc):
        # 1) делаем список токенов через regexp
        # 2) делаем стэмминг корней слов
        # 3) фильтруем полученные леммы через стоп слова
        # 4) возвращаем список
        regex_num_ponctuation = punctuation

        resval = [self.stemer.stem(t) for t in re.split(regex, doc.lower())
                if t not in regex_num_ponctuation and len(t) >= model_params['t_min_len']]

        global text_pbar
        text_pbar.update(1)

        return resval


from pymystem3 import Mystem

class LemmaTokenizer():
    # Лемматайзер приводит слова к неопределенному виду (сохраняется некоторый контекст)
    # почти в два раза медленнее и на ~0,001 дает точности больше чем стэмер
    # не пиклится :(
    def __init__(self):
        self.mystem = Mystem()

    def __call__(self, doc):
        # 1) делаем список токенов через regexp
        # 2) делаем лемминг слов
        # 3) фильтруем полученные леммы через стоп слова
        # 4) возвращаем список

        resval = []
        for t in re.split(regex, doc.lower()):
            lemma = self.mystem.lemmatize(t)  # позвращает список лемм
            # tokens.extend(lemma)
            for tt in lemma:
                if tt not in r_num_ponctuation and len(tt) >= model_params['t_min_len']:
                    resval.append(tt)

        global text_pbar
        text_pbar.update(1)

        return resval

[nltk_data] Downloading package stopwords to /home/kit-
[nltk_data]     kat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kit-kat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
# делаем словарик позиционный - энкодер (для всех каталогов)
# Планируемый классификатор может предсказывать любой каталога при наличии данных в трейн сете.
# Использовать будем только в конце. Лишние пустые категории нам в трейне щас не нужны.
catalogs = sorted(list(DG.nodes))
cat2id = {}
for i, cat in enumerate(catalogs):
    cat2id[cat] = i

In [36]:
# бинарные кодеры\декодеры
def dummify_labels(df_in, column, column_dict, catalogs):
    """
    разбивает столбец датафрейма на бинарные столбцы из списка. Просто принцип onehot
    """
    y = df_in[column].copy()
    # ушли в "примитивы", поиск по пандасу значительно дольше.
    out = np.zeros((len(y.index), len(catalogs)), dtype=float)

    for i, cat in enumerate(y[column[0]].values):
        cat_position = column_dict[cat]
        out[i][cat_position] = 1

    out = pd.DataFrame(out, index=y.index, columns=catalogs).astype(int)

    return out

def _dummify_labels(df_in, column, column_dict, catalogs):
    """
    разбивает столбец датафрейма на бинарные столбцы из списка предков по кажому элементу
    Используем только для подсчета иерархического F
    """
    y = df_in[column].copy()
    # ушли в "примитивы", поиск по пандасу значительно дольше.
    out = np.zeros((len(y.index), len(catalogs)), dtype=float)

    for i, cat in enumerate(y[model_params['labels'][0]].values):
        branch = all_predcestors(DG, cat)
        for leaf in branch:
            if leaf != -1:
                cat_position = column_dict[leaf]
                out[i][cat_position] = 1

    out = pd.DataFrame(out, index=y.index, columns=catalogs)

    return out

def undummify_labels(df_dimmies, column, column_list):
    """
    возвращает бинарные столбцы таргета в исходный один столбец.
    """
    y = df_dimmies.to_numpy()
    out = np.argmax(y, axis=1)
    out = np.array([column_list[i] for i in out])

    return pd.DataFrame(out, columns=column, index=df_dimmies.index)

In [37]:
# Все числовые параметры моделей подобраны.
# GridSearch делать не стал. Все в ручную, т.к. слишком много трансформаций при валидации
# Машина позволяет запускать по 10 тетрадок одновременно

# random_seed = np.random.randint(0,2**23)
random_seed = 262898
model_params = {'text': ['title', 'short_description', 'name_value_characteristics'],
                'numeric': ['rating'],
                'labels': ['category_id'],
                'random_split':{'test_size': 0.25,
                                'random_state': 42},
                't_min_len': 2,
                'vectorizer': {'ngram_range': (1, 2),
                               'max_features': 90_000},
                'hist_bin_edges': hist_bin_edges,
                'tree':
                    {'criterion':'entropy',
                     'max_depth':8,
                     'random_state':random_seed,
                    },
                # 'r_forest':
                #     {'n_estimators':100,
                #      'criterion':'entropy',
                #      'max_depth':8,
                #      'random_state':random_seed,
                #     },
                # 'catboost':{
                #     'loss_function':'CrossEntropy',
                #     'custom_metric':'AUC',
                #     'iterations':300,
                #     'depth':8,
                #     'learning_rate':0.01,
                #     'metric_period':10,
                #     'verbose':100,
                #     # 'silent':True,
                #     'early_stopping_rounds':20,
                #     'train_dir':'catboost_train_logs/',
                #     'task_type':'GPU',
                #     }
                }

In [38]:
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier


# Пайплайн
pl = Pipeline([
    ('union', FeatureUnion(
        # Объединяем подготовленные данные в спарс вектор
        transformer_list=[

            # Подготоваливаем текстовую часть данных
            ('text_features', Pipeline([
                # Выбрали только текстовые столбцы
                ('text_selector', FunctionTransformer(get_text_data_func, validate=False)),
                # Запустили прогрессбар
                ('text_pbar_start', FunctionTransformer(start_text_pbar, validate=False)),
                # создали спарс матрицу по словам(токенам)
                ('vectorizer', CountVectorizer(
                    tokenizer=StemTokenizer(),
                    # tokenizer=LemmaTokenizer(), # -результат хуже и не пиклится
                    **model_params['vectorizer'],
                                               )
                 ),
                # Всего уникальных слов ~ 88_000 (4 и более символа)
                # токенов со стэмером ~ 52_000 (экономия 40%)
                #
                # убили прогресс бар
                ('text_pbar_shut', FunctionTransformer(shut_text_pbar, validate=False)),
            ])),

            # Ухудшает прогноз...
            # Подготоваливаем цифровую часть данных
            # ('numeric_features', Pipeline([
            #     # Запустили прогрессбар
            #     ('num_pbar_start', FunctionTransformer(start_num_pbar, validate=False)),
            #     # Выбрали только числовые столбцы
            #     ('num_selector', FunctionTransformer(get_numeric_data_func, validate=False)),
            #     # убили прогрессбар
            #     ('num_pbar_shut', FunctionTransformer(shut_num_pbar, validate=False)),
            #     # подготовка, проверка числовых данных
            #     ('imputer', SimpleImputer())
            # ])),
        ]
    )),
    # Сама модель
    # """
    # Используем OneVsRestClassifier. Внутри идет итератор по каждому столбцу таргета...     
    # В текущем случае, мы имеем полный список "негативов" для всей модели. 
    # Но мы теряем, в своей метрике.
    # Не получится вытащить id столбца на этапе валидации... модель учится в отдельном пуле.
    # Также нет прогрессбара... :(
    # """
    ('clf', OneVsRestClassifier(DecisionTreeClassifier(**model_params['tree']))), #  0,92 топ
    # ('clf', OneVsRestClassifier(RandomForestClassifier(**model_params['r_forest']))),  # меньше точность 0,85
    # ("catboost", OneVsRestClassifier(
    #     estimator=CatBoostClassifier(**model_params['catboost']
    #         )
    #     )
    # )  # Катбуст тут не взлетел вообще. ~ 0.8 точность.
],
verbose=True)

In [39]:
# Матчинг - это задача классификации
# к таргетам добавляются специальные негативные метки (категории в которые id не входит)

# Предвидя проблему взрыва размерности, 
# добавлением новых строк и хранением одних и техже описаний товара текстом.

# я решил идти по пути OVR (OneVsRest) -- мы по сути вместо одной большой МОДЕЛИ,
# будем иметь 1477 маленьких моделей на каждый класс (бинарную колонку)

# Немного экономим в вычислениях.
# у нас в трейн сете размечено 1477 из 3327 категорий (экономия ~ 44%)
# поэтому бинаризацию категорий будем и делать по той разметке, что есть...
train_catalogs = set()
for cat in train['category_id']:
    train_catalogs.update(set(all_predcestors(DG, cat)))
train_catalogs.remove(-1)
train_catalogs = sorted(list(train_catalogs))

train_cat2id = {}
for i, cat in enumerate(train_catalogs):
    train_cat2id[cat] = i

catalog_cnt = len(train_catalogs)
print(f'в обучении будем использовать {catalog_cnt} категорий')

в обучении будем использовать 1477 категорий


In [41]:
# Дробим датасет на трейн и тест + "бинаризуем"
from sklearn.model_selection import train_test_split

# train = train[:100]
_train, _val = train_test_split(train, **model_params['random_split'])

X_train = _train[model_params['text'] + model_params['numeric']]
y_train = _train[model_params['labels']]

y_train_dummified = dummify_labels(y_train, model_params['labels'], train_cat2id, train_catalogs)


X_val = _val[model_params['text'] + model_params['numeric']]
y_val = _val[model_params['labels']]

y_val_dummified = dummify_labels(y_val, model_params['labels'], train_cat2id, train_catalogs)

In [42]:
# 1477 категорий
# ETA ~28.0min...
pl.fit(X=X_train,
       y=y_train_dummified
       )

Text processing: 100%|██████████| 212589/212589 [01:39<00:00, 2141.23it/s]


[Pipeline] ............. (step 1 of 2) Processing union, total= 1.7min
[Pipeline] ............... (step 2 of 2) Processing clf, total=14.9min


Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('text_features',
                                                 Pipeline(steps=[('text_selector',
                                                                  FunctionTransformer(func=<function get_text_data_func at 0x7f23ff458550>)),
                                                                 ('text_pbar_start',
                                                                  FunctionTransformer(func=<function start_text_pbar at 0x7f23ff007ee0>)),
                                                                 ('vectorizer',
                                                                  CountVectorizer(max_features=90000,
                                                                                  ngram_range=(1,
                                                                                               2),
                                                                                

In [43]:
# ETA ~9.0min...
result = pl.predict_proba(X=X_val)

Text processing: 100%|██████████| 70863/70863 [00:32<00:00, 2166.27it/s]


In [44]:
# сохраняем результаты обучения
from joblib import dump
dump(pl, 'pl.pkl')
dump(DG, 'DG.pkl')
dump(model_params, 'model_params.pkl')
dump(catalogs, 'catalogs.pkl')
dump(cat2id, 'cat2id.pkl')
dump(train_catalogs, 'train_catalogs.pkl')
dump(train_cat2id, 'train_cat2id.pkl')
# dump(X_val, 'X_val.pkl')
# dump(y_val, 'y_val.pkl')
# dump(result, 'result.pkl')

In [45]:
# загружаем результаты обучения
from joblib import load
# pl = load('pl.pkl')

# DG = load('DG.pkl')
# model_params = load('model_params.pkl')
# catalogs = load('catalogs.pkl')
# cat2id = load('cat2id.pkl')
# train_catalogs = load('train_catalogs.pkl')
# train_cat2id = load('train_cat2id.pkl')
# X_val = load('X_val.pkl')
# y_val = load('y_val.pkl')
# result = load('result.pkl')

In [46]:
result = pd.DataFrame(result, index=X_val.index, columns=train_catalogs)
result = undummify_labels(result, model_params['labels'], train_catalogs)

In [47]:
categories_maper = {}
for v in categories_tree.values:
    categories_maper[v[0]] = v[1]

compare_view = X_val.copy()
compare_view[model_params['labels'][0]] = result[model_params['labels'][0]]
compare_view['decriprion'] = compare_view[model_params['labels'][0]].map(categories_maper)


In [48]:
compare_view.sample(20)

,title,short_description,name_value_characteristics,rating,category_id,decriprion
227003,"Полка для ванной прямая//33*12,5*21 см/цвет бе...",None,None,5.000000,14149,Полки кухонные
103058,Гель лак LuckyLak Milky Way №01 8g,Не течет и хорошо самовыравнивается. Время пол...,None,4.753846,13995,Гель-лаки
136277,"Набор из 5 браслетов ""Мост"" с жемчугом, зеленый",None,None,0.000000,11574,Браслеты
143617,Новогодние салфетки бумажные 24х24 с рисунком...,None,None,0.000000,12730,Праздничный декор
273256,"Воздушный шар ""ЛОЛ"", 70 см",None,None,0.000000,12604,Воздушные шары и аксессуары
104318,Для специй. Бокс для хранения специй,None,None,0.000000,13901,Емкости для специй и мельницы
55819,"Карандаш для бровей Maybelline New York ""Brow ...","Карандаш для бровей ""Brow Ultra Slim"", каранда...",None,5.000000,14075,Карандаши для бровей
65014,"Парик ""Принцесса блондинка""",None,None,0.000000,2750,Парики и аксессуары
78302,Кабель AUX Разветвитель такневый металлический...,None,None,4.600000,12602,Разветвители
58674,ECO Organic greek figs Очищающий скраб для тел...,None,None,0.000000,12330,Скрабы и пилинги


In [49]:
# Оцениваем
from hF import h_fbeta_score

result[model_params['labels'][0]] = result[model_params['labels'][0]].apply(lambda x: x if x in train_cat2id.keys() else 0)
_result_dummified = _dummify_labels(result, model_params['labels'], cat2id, catalogs)
_target_dummified = _dummify_labels(y_val, model_params['labels'], cat2id, catalogs)


h_fbeta = h_fbeta_score(_target_dummified, _result_dummified)

print("h_fbeta_score: ", h_fbeta)

h_fbeta_score:  0.9207586205241317


In [50]:
dump(pl, '{0}_pl.pkl'.format(int(h_fbeta*1_000_000)))
dump(model_params, '{0}_model_params.pkl'.format(int(h_fbeta*1_000_000)))

['920758_model_params.plk']

In [51]:
model_params

{'text': ['title', 'short_description', 'name_value_characteristics'],
 'numeric': ['rating'],
 'labels': ['category_id'],
 'random_split': {'test_size': 0.25, 'random_state': 42},
 't_min_len': 2,
 'vectorizer': {'ngram_range': (1, 2), 'max_features': 90000},
 'hist_bin_edges': [0.0,
  0.8333333333333334,
  1.6666666666666667,
  2.5,
  3.3333333333333335,
  4.166666666666667,
  5.0],
 'tree': {'criterion': 'entropy', 'max_depth': 8, 'random_state': 262898}}

In [52]:
exit(0)